<a href="https://colab.research.google.com/github/JennyPre/ReinforcementSimple/blob/main/AutonomousCar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
! pip install gymnasium
! pip install gymnasium stable-baselines3
! pip install mediapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.9 MB/s eta 0:00:00


In [34]:
import gymnasium as gym
import torch.nn as nn
import torch
import mediapy as media
import numpy as np

In [41]:
env = gym.make('MountainCarContinuous-v0', render_mode='rgb_array')

# run a few episodes with random actions
frames = []
observation, info = env.reset()
rewards = []
#  Observation space: Box([-1.2 -0.07], [0.6 0.07], (2,), float32)

for t in range(10000):
  action = env.action_space.sample()
  frames.append(env.render())
  observation, reward, terminated, truncated, info = env.step(action)
  done = terminated or truncated # is termianted, when the car has position 0.45
  rewards.append(reward)
  if done:
    print(f"Episode finished after {t+1} timesteps. Cummulative return: {sum(rewards)}")
    break

media.show_video(frames, fps=50, loop=False)

Episode finished after 999 timesteps. Cummulative return: -33.06513361905771


This browser does not support the video tag.


In [ ]:
class Network(nn.Module):
    def __init__(self, state_size, action_size):
      super(Network, self).__init__()

      # state_size=2 (position, velocity)
      # action_size=1 (acceleration)

      self.fc1 = nn.Linear(state_size, 128)  # First layer: 2 -> 128
      self.fc2 = nn.Linear(128, 128)         # Hidden layer: 128 -> 128
      self.fc3 = nn.Linear(128, action_size) # Output layer: 128 -> 1

    def forward(self, x):
        """Pass the state through the network to get Q-values for each action"""

        l1 = self.fc1(x)
        l1 = torch.relu(l1)

        l2 = self.fc2(l1)
        l2 = torch.relu(l2)

        return self.fc3(l2)

In [5]:
# actions

state_size = env.observation_space.shape[0]
action_size = env.action_space.shape[0]
agent = "TODO"


In [ ]:
# train
# optimizer
# prediction
# visualization